### 목록 페이지에서 퍼머 링크 목록 추출

In [3]:
import requests
import lxml.html

In [4]:
response = requests.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
root = lxml.html.fromstring(response.content)
for a in root.cssselect('.view_box a'):
    url = a.get('href')
    print(url)

/store/books/look.php?p_code=B9923613168
javascript:;
/store/books/look.php?p_code=B9923613168
/store/books/look.php?p_code=B9037188841
/store/books/look.php?p_code=B9037188841
/store/books/look.php?p_code=B8955111301
javascript:;
/store/books/look.php?p_code=B8955111301
/store/books/look.php?p_code=B6758255857
javascript:;
/store/books/look.php?p_code=B6758255857
/store/books/look.php?p_code=B9737048306
/store/books/look.php?p_code=B9737048306
/store/books/look.php?p_code=B9009516784
/store/books/look.php?p_code=B9009516784
/store/books/look.php?p_code=B7533828722
/store/books/look.php?p_code=B7533828722
/store/books/look.php?p_code=B1833400260
/store/books/look.php?p_code=B1833400260
/store/books/look.php?p_code=B7970422863
javascript:;
/store/books/look.php?p_code=B7970422863
/store/books/look.php?p_code=B6993093151
/store/books/look.php?p_code=B6993093151
/store/books/look.php?p_code=B8128498093
javascript:;
/store/books/look.php?p_code=B8128498093
/store/books/look.php?p_code=B703

---

In [5]:
response = requests.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
root = lxml.html.fromstring(response.content)

In [6]:
# 모든 링크를 절대 URL로 변환
root.make_links_absolute(response.url)

# 목록에서 javascript 제거
for a in root.cssselect('.view_box .book_tit a'):
    url = a.get('href')
    print(url)

https://www.hanbit.co.kr/store/books/look.php?p_code=B9923613168
https://www.hanbit.co.kr/store/books/look.php?p_code=B9037188841
https://www.hanbit.co.kr/store/books/look.php?p_code=B8955111301
https://www.hanbit.co.kr/store/books/look.php?p_code=B6758255857
https://www.hanbit.co.kr/store/books/look.php?p_code=B9737048306
https://www.hanbit.co.kr/store/books/look.php?p_code=B9009516784
https://www.hanbit.co.kr/store/books/look.php?p_code=B7533828722
https://www.hanbit.co.kr/store/books/look.php?p_code=B1833400260
https://www.hanbit.co.kr/store/books/look.php?p_code=B7970422863
https://www.hanbit.co.kr/store/books/look.php?p_code=B6993093151
https://www.hanbit.co.kr/store/books/look.php?p_code=B8128498093
https://www.hanbit.co.kr/store/books/look.php?p_code=B7030127165
https://www.hanbit.co.kr/store/books/look.php?p_code=B9949658683
https://www.hanbit.co.kr/store/books/look.php?p_code=B7628794939
https://www.hanbit.co.kr/store/books/look.php?p_code=B7123382765
https://www.hanbit.co.kr/

---

* URL 목록 추출을 위한 scrape_list_page() 함수 정의

* scrpae_list_page() 함수의 반환값은 list처럼 반복 가능한 제너레이터로 구현(응답을 하나씩 줌)

In [8]:
def main():
    """
    크롤러의 메인 처리
    """
    # 여러 페이지에서 크롤링을 위해 Session 사용
    session = requests.Session()  
    # scrape_list_page() 함수를 호출해서 제너레이터를 추출
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    # 제너레이터는 list처럼 사용 가능
    for url in urls:
        print(url)
        print('-'*70)

In [9]:
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect('.view_box .book_tit a'):
        url = a.get('href')
        # yield 구문으로 제너레이터의 요소 반환
        yield url

In [10]:
if __name__ == '__main__':
    main()

https://www.hanbit.co.kr/store/books/look.php?p_code=B9923613168
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B9037188841
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B8955111301
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B6758255857
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B9737048306
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B9009516784
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B7533828722
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_

### 상세 페이지 스크래핑

In [11]:
def main():
    # 여러 페이지에서 크롤링을 위해 Session 사용
    session = requests.Session()  
    # scrape_list_page() 함수를 호출해서 제너레이터를 추출
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    for url in urls:
        response = session.get(url)  # Session을 사용해 상세 페이지를 추출
        ebook = scrape_detail_page(response)  # 상세 페이지에서 상세 정보를 추출
        print(ebook)  # 상세 정보 출력
        break  

In [12]:
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect('.view_box .book_tit a'):
        url = a.get('href')
        yield url

In [13]:
def scrape_detail_page(response):
    """f
    상세 페이지의 Response에서 책 정보를 dict로 추출
    """
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url': response.url,
        'title': root.cssselect('.store_product_info_box h3')[0].text_content(),
        'price': root.cssselect('.pbr strong')[0].text_content(),
        'content': [p.text_content()\
            for p in root.cssselect('#tabs_3 .hanbit_edit_view p')]
    }
    return ebook

In [14]:
if __name__ == '__main__':
    main()

{'url': 'https://www.hanbit.co.kr/store/books/look.php?p_code=B9923613168', 'title': '모바일 UX/UI 디자인 강의 with Adobe XD', 'price': '23,400', 'content': ['', '\r\n\t\t\tPART 01. 모바일 UX/UI 디자인\r\n', '\xa0', '\r\n\t\t\tCHAPTER 01. 모바일 앱 UX의 시작\xa0\r\n', 'LESSON 01. 모바일 앱과 UX 디자인의 발전\xa0', 'UX 디자인의 등장\xa0', '화면 크기의 변화와 UX의 발전', '실무에서 말하는 모바일 UX 디자인', '\r\n\t\t\tLESSON 02. UX 디자인을 완성하는 세 가지 요소\r\n', 'UX(User Experience) 이해하기', '하나. UX는 발명이 아닌 발견이다', '둘. UX는 편리함보다 익숙함을 따른다', '셋. UX는 제품이 아닌 가치를 제공한다', '\r\n\t\t\tLESSON 03. 직관적인 UX 디자인 요소\r\n', 'BX(Brand Experience)', 'UI(User Interface)', 'GUI(Graphical User Interface)', '인포그래픽(Infographics)', '인터랙션(Interaction)', '\xa0', '\r\n\t\t\tCHAPTER 02. 모바일 UX 시나리오\r\n', '\r\n\t\t\tLESSON 01. 모바일 UX 시나리오 의미와 과정\r\n', '모바일 UX 시나리오 과정', '\r\n\t\t\tLESSON 02. 서비스 구분하고 제작 방향 설정하기\r\n', '모바일 서비스 구분하기', '네이티브 앱 (Native App)\xa0', '모바일 웹앱 (Mobile WebApp)\xa0', '반응형 웹앱(RWD : Responsive Web Design)\xa0', '적응형 웹앱(AWD : Adaptive Web Design)\xa0', '하이브리드 앱(Hybrid Ap

---

* 목차에 포함돼 있는 공백을 제거할 수 있는 normalize_space() 함수 정의

* List Comprehension 구문에 조건을 추가해서 빈 문자열 제거

In [15]:
import re

In [16]:
def main():
    # 여러 페이지에서 크롤링을 위해 Session 사용
    session = requests.Session()  
    # scrape_list_page() 함수를 호출해서 제너레이터를 추출
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    for url in urls:
        response = session.get(url)  # Session을 사용해 상세 페이지를 추출
        ebook = scrape_detail_page(response)  # 상세 페이지에서 상세 정보를 추출
        print(ebook)  # 상세 정보 출력
        break  

In [17]:
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect('.view_box .book_tit a'):
        url = a.get('href')
        yield url

In [18]:
def scrape_detail_page(response):
    """
    상세 페이지의 Response에서 책 정보를 dict로 추출
    """
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url': response.url,
        'title': root.cssselect('.store_product_info_box h3')[0].text_content(),
        'price': root.cssselect('.pbr strong')[0].text_content(),
        'content': [normalize_spaces(p.text_content())
            for p in root.cssselect('#tabs_3 .hanbit_edit_view p')
            if normalize_spaces(p.text_content()) != '']
    }
    return ebook

In [19]:
def normalize_spaces(s):
    """
    연결된 공백을 하나의 공백으로 변경
    """
    return re.sub(r'\s+', ' ', s).strip()

In [21]:
if __name__ == '__main__':
    main()

{'url': 'https://www.hanbit.co.kr/store/books/look.php?p_code=B9923613168', 'title': '모바일 UX/UI 디자인 강의 with Adobe XD', 'price': '23,400', 'content': ['PART 01. 모바일 UX/UI 디자인', 'CHAPTER 01. 모바일 앱 UX의 시작', 'LESSON 01. 모바일 앱과 UX 디자인의 발전', 'UX 디자인의 등장', '화면 크기의 변화와 UX의 발전', '실무에서 말하는 모바일 UX 디자인', 'LESSON 02. UX 디자인을 완성하는 세 가지 요소', 'UX(User Experience) 이해하기', '하나. UX는 발명이 아닌 발견이다', '둘. UX는 편리함보다 익숙함을 따른다', '셋. UX는 제품이 아닌 가치를 제공한다', 'LESSON 03. 직관적인 UX 디자인 요소', 'BX(Brand Experience)', 'UI(User Interface)', 'GUI(Graphical User Interface)', '인포그래픽(Infographics)', '인터랙션(Interaction)', 'CHAPTER 02. 모바일 UX 시나리오', 'LESSON 01. 모바일 UX 시나리오 의미와 과정', '모바일 UX 시나리오 과정', 'LESSON 02. 서비스 구분하고 제작 방향 설정하기', '모바일 서비스 구분하기', '네이티브 앱 (Native App)', '모바일 웹앱 (Mobile WebApp)', '반응형 웹앱(RWD : Responsive Web Design)', '적응형 웹앱(AWD : Adaptive Web Design)', '하이브리드 앱(Hybrid App)', 'LESSON 03. 태스크 분석을 통한 주요 UI 설계하기', '태스크 분석(Task Research) 과정', 'LESSON 04. 카테고리 분석을 통한 디자인 포지션 찾기', '1단계. 유사 카테고리 서비스 찾아 분석하기', '2단계. 동일 사용자

---

* 전체 페이지 크롤링을 위해 main() 함수의 for 구문에서 break 제거

* 대상 서버에 부하를 주지 않은 상태에서 크롤링을 위해 time 모듈을 import 및 time.sleep(1)을 넣어 1초 동안 대기

In [22]:
import time

In [23]:
def main():
    # 여러 페이지에서 크롤링을 위해 Session 사용
    session = requests.Session()  
    # scrape_list_page() 함수를 호출해서 제너레이터를 추출
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    for url in urls:
        time.sleep(1) # 1초간 대기
        response = session.get(url)  # Session을 사용해 상세 페이지를 추출
        ebook = scrape_detail_page(response)  # 상세 페이지에서 상세 정보를 추출
        print(ebook)  # 상세 정보 출력
        break  

In [24]:
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect('.view_box .book_tit a'):
        url = a.get('href')
        yield url

In [25]:
def scrape_detail_page(response):
    """
    상세 페이지의 Response에서 책 정보를 dict로 추출
    """
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url': response.url,
        'title': root.cssselect('.store_product_info_box h3')[0].text_content(),
        'price': root.cssselect('.pbr strong')[0].text_content(),
        'content': [normalize_spaces(p.text_content())
            for p in root.cssselect('#tabs_3 .hanbit_edit_view p')
            if normalize_spaces(p.text_content()) != '']
    }
    return ebook

In [26]:
def normalize_spaces(s):
    """
    연결된 공백을 하나의 공백으로 변경
    """
    return re.sub(r'\s+', ' ', s).strip()

In [27]:
if __name__ == '__main__':
    main()

{'url': 'https://www.hanbit.co.kr/store/books/look.php?p_code=B9923613168', 'title': '모바일 UX/UI 디자인 강의 with Adobe XD', 'price': '23,400', 'content': ['PART 01. 모바일 UX/UI 디자인', 'CHAPTER 01. 모바일 앱 UX의 시작', 'LESSON 01. 모바일 앱과 UX 디자인의 발전', 'UX 디자인의 등장', '화면 크기의 변화와 UX의 발전', '실무에서 말하는 모바일 UX 디자인', 'LESSON 02. UX 디자인을 완성하는 세 가지 요소', 'UX(User Experience) 이해하기', '하나. UX는 발명이 아닌 발견이다', '둘. UX는 편리함보다 익숙함을 따른다', '셋. UX는 제품이 아닌 가치를 제공한다', 'LESSON 03. 직관적인 UX 디자인 요소', 'BX(Brand Experience)', 'UI(User Interface)', 'GUI(Graphical User Interface)', '인포그래픽(Infographics)', '인터랙션(Interaction)', 'CHAPTER 02. 모바일 UX 시나리오', 'LESSON 01. 모바일 UX 시나리오 의미와 과정', '모바일 UX 시나리오 과정', 'LESSON 02. 서비스 구분하고 제작 방향 설정하기', '모바일 서비스 구분하기', '네이티브 앱 (Native App)', '모바일 웹앱 (Mobile WebApp)', '반응형 웹앱(RWD : Responsive Web Design)', '적응형 웹앱(AWD : Adaptive Web Design)', '하이브리드 앱(Hybrid App)', 'LESSON 03. 태스크 분석을 통한 주요 UI 설계하기', '태스크 분석(Task Research) 과정', 'LESSON 04. 카테고리 분석을 통한 디자인 포지션 찾기', '1단계. 유사 카테고리 서비스 찾아 분석하기', '2단계. 동일 사용자